# 0a. Get pilot area 

In [ ]:
# Third-party library imports
import geopandas as gpd
from shapely.geometry import Polygon

# Local or project-specific imports
import settings as st

if st.my_run == "azure":
    import config_azure as cf
elif st.my_run == "local":
    import config as cf


In [ ]:
if st.my_run == "azure":
    os.system('sudo blobfuse /home/azureuser/cloudfiles/code/blobfuse/sidewalk --tmp-path=/mnt/resource/blobfusetmp --config-file=/home/azureuser/cloudfiles/code/blobfuse/fuse_connection_sidewalk.cfg -o attr_timeout=3600 -o entry_timeout=3600 -o negative_timeout=3600 -o allow_other -o nonempty')

## Get administrative areas of Amsterdam

For administrative purposes, the city is divided in pieces, on different levels: 'stadsdelen', 'gebieden', 'wijken' and 'buurten'. These can be viewed here: https://maps.amsterdam.nl/gebiedsindeling/. Data can be retrieved from an API: https://api.data.amsterdam.nl/v1/docs/datasets/gebieden.html

In [ ]:
# Get area data
if st.my_areas:
    area_url = "https://api.data.amsterdam.nl/v1/gebieden/" + st.area_choice + "/?_format=geojson" 
    gdf_area_raw = gpd.read_file(area_url)

In [ ]:
# Select columns of geodataframe
if st.my_areas:
    gdf_area = gdf_area_raw[['geometry', 'code', 'naam']]
    gdf_area = gdf_area.to_crs(st.CRS)
else:
    polygon = Polygon(st.coordinates)
    gdf_area = gpd.GeoDataFrame(geometry=[polygon], crs=st.CRS)
    gdf_area['naam'] = 'Part Osdorp-Midden'
    gdf_area['code'] = 'OM'

## Store output with selected areas 

In [ ]:
# Make a selection of areas
if st.my_areas:
    gdf_area_select = gdf_area[gdf_area['naam'].isin(st.my_areas)].reset_index(drop=True)
else:
    gdf_area_select = gdf_area

In [ ]:
# Store area geometries in a file
gdf_area_select.to_file(cf.output_pilot_area, driver='GeoJSON')